In [ ]:
# lading the python NWB API
import pynwb, os, sys
import numpy as np # numpy for numerical analysis
import matplotlib.pylab as plt # + matplotlib for vizualization
sys.path.append(os.path.join(os.path.expanduser('~'), 'work', 'physion', 'physion'))
from dataviz.show_data import MultimodalData # custom
from datavyz import graph_env_notebook as ge